In [ ]:
%load_ext autoreload
%autoreload 2
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import anndata as ad
mpl.rcParams['figure.dpi'] = 150
plt.rcParams['pdf.fonttype'] = 42
import sys
from spatial_analysis import *
from plotting import *
from utils import *

In [ ]:
import seaborn as sns
sns.set_style('white')

In [ ]:
def unbinarize_strings(A):
    try:
        A.var_names = [i.decode('ascii') for i in A.var_names]
        A.obs.index = [i.decode('ascii') for i in A.obs.index]
        for i in A.obs.columns:
            if A.obs[i].dtype != np.dtype('bool') and \
                A.obs[i].dtype != np.dtype('int64') and \
                A.obs[i].dtype != np.dtype('int32') and \
                A.obs[i].dtype != np.dtype('object_') and \
                A.obs[i].dtype != np.dtype('float64') and A.obs[i].dtype != np.dtype('float32'):
                if A.obs[i].dtype.is_dtype('category'):
                    try:
                        A.obs[i] = [i.decode('ascii') for i in A.obs[i]]
                    except Exception as e:
                        pass
    except Exception as e:
        print(e)
    return A


In [ ]:
# load annotated data (missing aging-related genes) and full data

In [ ]:
adata_annot = ad.read_h5ad("/faststorage/brain_aging/merfish/exported/011722_adata_combined_harmony.h5ad")
adata_annot = unbinarize_strings(adata_annot)
adata_annot = adata_annot[adata_annot.obs.dtype=='merfish']

In [ ]:
adata_annot.raw = unbinarize_strings(adata_annot.raw.to_adata())

In [ ]:
celltype_colors, celltype_pals, label_colors, clust_pals = generate_palettes(adata_annot)

# Compute neighborhoods

In [ ]:
adata_annot_young = adata_annot[adata_annot.obs.age=='4wk']
adata_annot_med = adata_annot[adata_annot.obs.age=='24wk']
adata_annot_old = adata_annot[adata_annot.obs.age=='90wk']


In [ ]:
clust_order = [
 'ExN-L2/3-1',
 'ExN-L2/3-2',
 'ExN-L5-1',
 'ExN-L5-2',
 'ExN-L5-3',
 'ExN-L6-1',
 'ExN-L6-2',
 'ExN-L6-3',
 'ExN-L6-4',
 'ExN-LatSept',

 'InN-Calb2',
 'InN-Chat',
 'InN-Lamp5',
 'InN-LatSept',
 'InN-Pvalb-1',
 'InN-Pvalb-2',
 'InN-Pvalb-3',
 'InN-Sst',
 'InN-Vip',
 'MSN-D1-1',
 'MSN-D1-2',
 'MSN-D2',
 'OPC',
 'Olig-1',
 'Olig-2',
 'Olig-3',

'Astro-1',
 'Astro-2',
 'Vlmc',
 'Peri-1',
 'Peri-2',
 'Endo-1',
 'Endo-2',
 'Endo-3',
 'Epen',

 'Micro-1',
 'Micro-2',
 'Micro-3',
 'Macro',
 'T cell',
]

In [ ]:
import multiprocessing
from joblib import Parallel, delayed

In [ ]:
young_neighbors, young_zscore, young_nbor_pvals = compute_celltype_neighborhood(adata_annot_young, 'cell_type',celltypes=clust_annots, niter=500, radius=150)

In [ ]:
med_neighbors, med_zscore, med_nbor_pvals = compute_celltype_neighborhood(adata_annot_med, 'clust_annot',celltypes=clust_annots, niter=500, radius=150)

In [ ]:
old_neighbors, old_zscore, old_nbor_pvals = compute_celltype_neighborhood(adata_annot_old, 'clust_annot', niter=500, celltypes=clust_annots, radius=150)

In [ ]:
# hierarchically cluster zscore
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

def hierarchical_cluster_order(mat, method='ward'):
    D = pdist(mat,'cosine')
    D[np.isnan(D)] = 0
    Z = linkage(D,method,optimal_ordering=True)
    den = dendrogram(Z, no_plot=True)
    return np.array(den['leaves'])

def clust_avg(A, clust_key, clust_names):
    return np.array([A[A.obs[clust_key]==i].X.mean(0) for i in clust_names])

In [ ]:

young_clust_avg = clust_avg(adata_annot_young, 'clust_annot', clust_annots)
clust_order = hierarchical_cluster_order(young_zscore, method='complete')
ex_clusts = np.argwhere([True if "Ex" in i else False for i in clust_annots]).flatten()
in_clusts = np.argwhere([True if ("In" in i or "MSN" in i) else False for i in clust_annots]).flatten()
nn_clusts = np.argwhere([True if ("Ex" not in i and "In" not in i and "MSN" not in i) else False for i in clust_annots]).flatten()

exn_order = hierarchical_cluster_order(old_zscore[ex_clusts,:][:, ex_clusts], 'complete')
in_order = hierarchical_cluster_order(old_zscore[in_clusts,:][:, in_clusts], 'complete')+exn_order.max()+1
nn_order = hierarchical_cluster_order(old_zscore[nn_clusts,:][:, nn_clusts], 'complete')+in_order.max()+1

clust_order = np.arange(len(clust_order))#np.hstack((exn_order, in_order, nn_order))

In [ ]:
seg_points = [9, 20, 26]
def plot_nborhood(zs, clust_order, hide_labels=False, seg_points=None,vmin=-200,vmax=200):
    f, ax = plt.subplots(figsize=(10,10))
    gs = plt.GridSpec(nrows=2,ncols=2,width_ratios=[0.5, 20], height_ratios=[20,0.5], wspace=0.01, hspace=0.01)

    ax = plt.subplot(gs[0,0])
    curr_cmap = mpl.colors.ListedColormap([label_colors[i] for i in np.array(clust_annots)[clust_order][::-1]])
    ax.imshow(np.expand_dims(np.arange(zs.shape[0]),1),aspect='auto',interpolation='none', cmap=curr_cmap,rasterized=True)
    sns.despine(ax=ax,bottom=True,left=True)
    if hide_labels:
        ax.set_yticks([]);
        ax.set_yticklabels([]);
        ax.set_xticks([])
    else:
        ax.set_yticks(np.arange(len(clust_annots)));
        ax.set_yticklabels(np.array(clust_annots)[clust_order][::-1]);
        ax.set_xticks([])

    ax = plt.subplot(gs[0,1])
    ax.imshow(np.flipud(zs[clust_order,:][:,clust_order]),aspect='auto',interpolation='none',vmin=vmin,vmax=vmax,cmap=plt.cm.seismic, rasterized=True)
    ax.axis('off')
    if seg_points is not None:
        for i in seg_points:
            ax.axvline(i-0.5,color='k',linestyle='--')
            ax.axhline(len(clust_annots)-i-0.5,color='k',linestyle='--')

    ax = plt.subplot(gs[1,1])
    ax.imshow(np.expand_dims(np.arange(zs.shape[1])[::-1],1).T,aspect='auto',interpolation='none',cmap=curr_cmap,rasterized=True)
    sns.despine(ax=ax,bottom=True,left=True)
    if hide_labels:
        ax.set_xticks([])
        ax.set_yticks([])
    else:
        ax.set_xticks(np.arange(len(clust_annots)));
        ax.set_yticks([])
        ax.set_xticklabels(np.array(clust_annots)[clust_order],rotation=90);
    return f


# Fig. 3: Neighborhood and interaction analysis

In [ ]:
def nearest_neighbor_dists(A, cell_type_source, cell_type_targets):
    """ Compute nearest neighbor distances from source to targets """
    pass

In [ ]:
old_pos = adata_annot[adata_annot.obs.age=='90wk'].obsm['spatial']
old_nn, _ = KDTree(old_pos).query(old_pos, k=2)


In [ ]:
# look at gross distances to nearest neighbors
from sklearn.neighbors import KDTree
old_pos = adata_annot[adata_annot.obs.age=='90wk'].obsm['spatial']
old_nn, _ = KDTree(old_pos).query(old_pos, k=2)
old_nn = old_nn[:,1]

med_pos = adata_annot[adata_annot.obs.age=='24wk'].obsm['spatial']
med_nn, _ = KDTree(old_pos).query(old_pos, k=2)
med_nn = med_nn[:,1]

young_pos = adata_annot[adata_annot.obs.age=='4wk'].obsm['spatial']
young_nn, _ = KDTree(young_pos).query(young_pos, k=2)
young_nn = young_nn[:,1]


In [ ]:
#celltypes = adata_annot.obs.remapped_cell_type.unique()
celltypes = [
    'InN',
 'ExN',
 'MSN',
 'Astro',
 'OPC',
 'Olig',
 'Endo',
 'Vlmc',
 'Peri',
 'Macro',
 'Micro',
]


In [ ]:
adata_annot.obs['clust_reduced'] = ["-".join(i.split('-')[:-1]) if len(i.split('-'))>1 else i for i in adata_annot.obs.clust_annot]

In [ ]:
clust_reduced_labels = list(adata_annot.obs.clust_reduced.unique())

In [ ]:
niter = 500
perturb_max = 100
dist_thresh = 20

In [ ]:
#celltypes = sorted(adata_annot.obs.cell_type.unique())
young_interactions_clust, young_pvals_clust, young_qvals_clust = compute_celltype_interactions(adata_annot[adata_annot.obs.age=='4wk'], 
                                                                'cell_type', celltypes,niter=niter,dist_thresh=dist_thresh,perturb_max=perturb_max)


In [ ]:
med_interactions_clust, med_pvals_clust, med_qvals_clust = compute_celltype_interactions(adata_annot[adata_annot.obs.age=='24wk'], 
                                                                'cell_type', celltypes,niter=niter,dist_thresh=dist_thresh,perturb_max=perturb_max)


In [ ]:
old_interactions_clust, old_pvals_clust, old_qvals_clust = compute_celltype_interactions(adata_annot[adata_annot.obs.age=='90wk'], 
                                                                'cell_type', celltypes,niter=niter,dist_thresh=dist_thresh,perturb_max=perturb_max)


In [ ]:
from statsmodels.stats.multitest import multipletests
def fdr_correct(X):
    new_X = np.zeros_like(X)
    for i in range(X.shape[0]):
        pvals = multipletests(X[i,:],method='fdr_bh')[1]
        new_X[i,:] = multipletests(X[i,:],method='fdr_bh')[1]
        new_X[:,i] = new_X[i,:]
    #X = multipletests(X.flatten(), method='fdr_bh')[1]
    return new_X#X.reshape(X_shape)

In [ ]:
young_qvals_clust = fdr_correct(young_pvals_clust.copy())
med_qvals_clust = fdr_correct(med_pvals_clust.copy())
old_qvals_clust = fdr_correct(old_pvals_clust.copy())


In [ ]:
young_interactions_clust[np.isinf(young_interactions_clust)] = 5
med_interactions_clust[np.isinf(med_interactions_clust)] = 5
old_interactions_clust[np.isinf(old_interactions_clust)] = 5


In [ ]:
young_qvals_clust[np.isnan(young_qvals_clust)] = 1
med_qvals_clust[np.isnan(med_qvals_clust)] = 1
old_qvals_clust[np.isnan(old_qvals_clust)] = 1

In [ ]:
from plotting import plot_interactions

In [ ]:
f = plot_interactions(young_qvals_clust, young_interactions_clust, celltypes,celltype_colors,cmap=plt.cm.seismic,vmax=5, vmin=-5)
f.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_cell_contact_young.pdf",bbox_inches='tight', dpi=200)

In [ ]:
f = plot_interactions(med_qvals_clust, med_interactions_clust, celltypes,celltype_colors,cmap=plt.cm.seismic,vmax=3, vmin=-3)
f.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_cell_contact_med.pdf",bbox_inches='tight', dpi=200)

In [ ]:
f = plot_interactions(old_qvals_clust, old_interactions_clust, celltypes,celltype_colors,cmap=plt.cm.seismic,vmax=3, vmin=-3)
f.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_cell_contact_old.pdf",bbox_inches='tight', dpi=200)

In [ ]:
diff = old_interactions_clust-young_interactions_clust
diff[np.isnan(diff)] = 0
for i in range(diff.shape[0]):
    for j in range(diff.shape[1]):
        if young_qvals_clust[i,j] < 0.05 or old_pvals_clust[i,j] < 0.05:
            pass
        else:
            diff[i,j] = 0
diff_qvals = np.zeros_like(old_qvals_clust)
for i in range(old_qvals_clust.shape[0]):
    for j in range(old_qvals_clust.shape[0]):
        if old_qvals_clust[i,j] < 0.05 or young_qvals_clust[i,j] < 0.05:
            if (old_interactions_clust[i,j]>0 or young_interactions_clust[i,j]>0):
                diff_qvals[i,j] = 0
            else:
                diff_qvals[i,j] = 1
        else:
            diff_qvals[i,j] = 1

In [ ]:
f = plot_interactions(diff_qvals, diff, celltypes,celltype_colors,cmap=plt.cm.Reds,vmax=1.2, vmin=0)
f.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_cell_contact_diff.pdf",bbox_inches='tight', dpi=200)

In [ ]:
#f,ax = plt.subplots(figsize=(5,5))
#ax.imshow(np.flipud(diff),cmap=plt.cm.seismic,vmin=-2.5, vmax=2.5)
#ax.set_xticks(np.arange(diff.shape[0]));
#ax.set_xticklabels(celltypes,rotation=90)
#ax.set_yticks(np.arange(diff.shape[1]));
#ax.set_yticklabels(celltypes[::-1]);
#sns.despine(ax=ax,left=True, bottom=True)
#f.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_cell_contact_diff.pdf",bbox_inches='tight', dpi=200)

# Redo this at higher resolution

# test effect of cell-cell interaction on activated state

In [ ]:
sc.tl.score_genes(adata_annot, gene_list=['B2m','Trem2', 'Ccl2', 'Apoe',  'Axl', 'Itgax', 'Cd9','C1qa','C1qc','Lyz2','Ctss'], score_name='activate_micro', use_raw=False)
sc.tl.score_genes(adata_annot, gene_list=['C4b', 'C3', 'Serpina3n', 'Cxcl10', 'Gfap', 'Vim', 'Il18','Hif3a'], score_name='activate_astro', use_raw=False)

sc.tl.score_genes(adata_annot, gene_list=activate_endo, score_name='activate_endo',use_raw=False)

In [ ]:
adata_micro = adata_annot[adata_annot.obs.cell_type=="Micro"]
adata_annot.obs.activate_micro = adata_annot.obs.activate_micro - np.mean(adata_micro[adata_micro.obs.age=='4wk'].obs.activate_micro)

In [ ]:
adata_astro = adata_annot[adata_annot.obs.cell_type=="Astro"]
adata_annot.obs.activate_astro = adata_annot.obs.activate_astro - np.mean(adata_astro[adata_astro.obs.age=='4wk'].obs.activate_astro)

In [ ]:
from spatial_analysis import *
from plotting import *

In [ ]:
sc.pl.umap(adata_annot, color=['age','activate_astro'])

In [ ]:
def identify_nearest_neighbors_with_idx(X,Y,dist_thresh, min_dist_thresh=15):
    if X.shape[0] > 0 and Y.shape[0] > 0:
        kdtree = KDTree(Y)
        ind, dists = kdtree.query_radius(X, r=dist_thresh, count_only=False,return_distance=True)
        ind_X = np.hstack([[i]*len(ind[i]) for i in np.arange(len(ind)) if len(ind[i])>0])
        
        ind = np.hstack(ind)
        dists = np.hstack(dists)
        if len(ind) > 0:
            ind = ind[dists>min_dist_thresh]      
            ind_X = ind_X[dists>min_dist_thresh]
        return ind.astype(np.int), ind_X.astype(np.int)
    else:
        return np.array([])

def count_neighbors_with_idx(X,Y,dist_thresh, ):
    if X.shape[0] > 0 and Y.shape[0] > 0:
        kdtree = KDTree(Y)
        ind, dists = kdtree.query_radius(X, r=dist_thresh, count_only=False,return_distance=True)
        counts_Y = np.array([len(i) for i in ind])
        ind_X = np.arange(len(ind))#np.array([i for i in np.arange(len(ind)) if len(ind[i])>0])
        return ind_X.astype(np.int), counts_Y.astype(np.int)
    else:
        return np.array([])

def identify_nearest_neighbors_with_dist(X,Y, min_dist=0):
    if X.shape[0] > 0 and Y.shape[0] > 0:
        kdtree = KDTree(Y)
        dists, ind = kdtree.query(X, k=2,return_distance=True)
        print(dists.shape, ind.shape)
        good_dists = np.zeros(len(dists))
        good_ind = np.zeros(len(ind))
        for i in range(dists.shape[0]):
            if dists[i,0] > 0: # remove duplicates
                good_dists[i] = dists[i,0]
                good_ind[i] = ind[i,0]
            else:
                good_dists[i] = dists[i,1]
                good_ind[i] = ind[i,1]
        #ind_X = np.hstack([[i]*len(ind[i]) for i in np.arange(len(ind)) if len(ind[i])>0])
        return good_dists, good_ind
    else:
        return np.array([])
    
def compute_celltype_obs_count_correlation(A,cell_type_X, cell_type_Y, obs_key_X, celltype_key='cell_type',radius=40, min_dist_thresh=15):
    X = A[A.obs[celltype_key] == cell_type_X]
    Y = A[A.obs[celltype_key] == cell_type_Y]
    obs_X = X.obs[obs_key_X]
    curr_X = X.obsm['spatial']
    curr_Y = Y.obsm['spatial']
    ind_X, counts_Y = count_neighbors_with_idx(curr_X, curr_Y, dist_thresh=radius)
    return obs_X.values[ind_X], ind_X, counts_Y


def compute_celltype_obs_distance_correlation(A,cell_type_X, cell_type_Y, obs_key_X, celltype_key1='cell_type', celltype_key2='cell_type'):
    X = A[A.obs[celltype_key1] == cell_type_X]
    Y = A[A.obs[celltype_key2] == cell_type_Y]
    obs_X = X.obs[obs_key_X]
    curr_X = X.obsm['spatial']
    curr_Y = Y.obsm['spatial']
    dists_Y, ind_Y = identify_nearest_neighbors_with_dist(curr_X, curr_Y)
    return obs_X.values, dists_Y

def compute_celltype_obs_correlation(A,cell_type_X, cell_type_Y, obs_key_X, obs_key_Y, celltype_key='cell_type', radius=40, min_dist_thresh=15):
    X = A[A.obs[celltype_key] == cell_type_X]
    Y = A[A.obs[celltype_key] == cell_type_Y]
    obs_X = X.obs[obs_key_X]
    obs_Y = Y.obs[obs_key_Y]
    curr_X = X.obsm['spatial']
    curr_Y = Y.obsm['spatial']
    neighbors_X, ind_X = identify_nearest_neighbors_with_idx(curr_X, curr_Y, dist_thresh=radius, min_dist_thresh=min_dist_thresh)
    curr_expr = obs_Y[neighbors_X]
    return obs_X.values[ind_X], curr_expr.values

In [ ]:
def compute_binned_values(dists, scores, min_d=0, max_d=100, bin_size=30):
    binned_mean = np.zeros(max_d-min_d-bin_size)
    binned_std = np.zeros(max_d-min_d-bin_size)
    for i in np.arange(min_d, max_d-bin_size):
        # find distances in this bin range
        idx = np.argwhere(np.logical_and(dists>i, dists<=(i+bin_size)))
        curr_scores = scores[idx]
        binned_mean[i] = np.mean(curr_scores)#/len(idx)
        binned_std[i] = np.std(curr_scores)/np.sqrt(len(curr_scores))#/len(idx)
    binned_mean -= binned_mean.mean()
    binned_std -= binned_mean.mean()
    return binned_mean, binned_std

In [ ]:
# astro to peri-1/peri-2
plt.figure(figsize=(3,3))
celltypes = ["Peri-1","Peri-2"]
for i in celltypes:
    scores, dists = compute_celltype_obs_distance_correlation(adata_annot[adata_annot.obs.age=='4wk'], "Astro", i, "activate_astro", celltype_key2='clust_annot')
    binned_mean, binned_std = compute_binned_values(dists, scores,bin_size=30,max_d=80)
    x = np.arange(len(binned_mean))+30
    plt.plot(x,binned_mean,color=label_colors[i])
    plt.fill_between(x,binned_mean-binned_std, binned_mean+binned_std,alpha=0.1,color=label_colors[i])
#plt.legend( celltypes)
plt.ylim([-0.2, 0.3])
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_distance_peri_score_4wk.pdf",bbox_inches='tight',dpi=300)

In [ ]:
# astro to peri-1/peri-2
plt.figure(figsize=(3,3))
celltypes = ["Peri-1","Peri-2"]
for i in celltypes:
    scores, dists = compute_celltype_obs_distance_correlation(adata_annot[adata_annot.obs.age=='90wk'], "Astro", i, "activate_astro", celltype_key2='clust_annot')
    binned_mean, binned_std = compute_binned_values(dists, scores,bin_size=30,max_d=80)
    x = np.arange(len(binned_mean))+30
    plt.plot(x,binned_mean,color=label_colors[i])
    plt.fill_between(x,binned_mean-binned_std, binned_mean+binned_std,alpha=0.1,color=label_colors[i])
#plt.legend( celltypes)
plt.ylim([-0.2, 0.4])
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_distance_peri_score_90wk.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(3,3))
celltypes = ["Peri","Endo","Vlmc", "Olig"]
for i in celltypes:
    scores, dists = compute_celltype_obs_distance_correlation(adata_annot[adata_annot.obs.age=='4wk'], "Micro", i, "activate_micro")
    binned_mean, binned_std = compute_binned_values(dists, scores,bin_size=30)
    x = np.arange(len(binned_mean))+30
    plt.plot(x,binned_mean,color=celltype_colors[i])
    plt.fill_between(x,binned_mean-binned_std, binned_mean+binned_std,alpha=0.1,color=celltype_colors[i])
plt.legend( celltypes)
plt.ylim([-0.05, 0.12])
sns.despine()
#plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_distance_micro_score_4wk.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(3,3))
for i in celltypes:
    scores, dists = compute_celltype_obs_distance_correlation(adata_annot[adata_annot.obs.age=='90wk'], "Micro", i, "activate_micro")
    binned_mean, binned_std = compute_binned_values(dists, scores,bin_size=30)
    x = np.arange(len(binned_mean))+30
    plt.plot(x,binned_mean,color=celltype_colors[i])
    plt.fill_between(x,binned_mean-binned_std, binned_mean+binned_std,alpha=0.1,color=celltype_colors[i])

plt.legend( celltypes)
plt.ylim([-0.05, 0.12])
sns.despine()
#plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_distance_micro_score_90wk.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(3,3))
#celltypes = ["Endo","Vlmc", "Olig", "Micro"]
for i in celltypes:
    scores, dists = compute_celltype_obs_distance_correlation(adata_annot[adata_annot.obs.age=='4wk'], "Astro", i, "activate_astro")
    binned_mean, binned_std = compute_binned_values(dists, scores,bin_size=30,max_d=100)
    x = np.arange(len(binned_mean))+30
    plt.plot(x,binned_mean,color=celltype_colors[i])
    plt.fill_between(x,binned_mean-binned_std, binned_mean+binned_std,alpha=0.1,color=celltype_colors[i])
plt.legend( celltypes )
sns.despine()
plt.ylim([-0.2, 0.3])
#plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_distance_astro_score_4wk.pdf",bbox_inches='tight',dpi=300)

In [ ]:
plt.figure(figsize=(3,3))

for i in celltypes:
    scores, dists = compute_celltype_obs_distance_correlation(adata_annot[adata_annot.obs.age=='90wk'], "Astro", i, "activate_astro")
    binned_mean, binned_std = compute_binned_values(dists, scores,bin_size=30,max_d=150)
    x = np.arange(len(binned_mean))+30
    plt.plot(x,binned_mean,color=celltype_colors[i])
    plt.fill_between(x,binned_mean-binned_std, binned_mean+binned_std,alpha=0.1,color=celltype_colors[i])
#plt.legend(celltypes)
sns.despine()
#plt.ylim([-0.2, 0.3])
#plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_distance_astro_score_90wk.pdf",bbox_inches='tight',dpi=300)

In [ ]:
sc.tl.score_genes(adata_annot, gene_list=[ "C4b", "Il18", "Il33"], score_name="activate_olig",use_raw=False)

In [ ]:
adata_olig = adata_annot[adata_annot.obs.cell_type=="Olig"]
adata_annot.obs.activate_olig = adata_annot.obs.activate_olig - np.mean(adata_olig[adata_olig.obs.age=='4wk'].obs.activate_olig)

In [ ]:
x,y = compute_celltype_obs_correlation(adata_annot,  "Olig","Micro", f"activate_olig",f"activate_micro",   radius=40)


In [ ]:
spatial_regions = ["Pia", "L2/3", "L5", "L6", "CC", "Striatum", "Ventricle"]
ct_combos = [["Olig", "Astro"],["Olig","Micro"],["Micro", "Astro"]]
cc = np.zeros((len(spatial_regions), 3))
for i,r in enumerate(spatial_regions):
    print(r)
    for j,t in enumerate(ct_combos):
        t1 = t[0]
        t2 = t[1]
        #curr_annot = adata_annot[adata_annot.obs.age=='90wk']
        x,y = compute_celltype_obs_correlation(adata_annot[adata_annot.obs.spatial_clust_annots==r],  t1,t2, f"activate_{t1.lower()}",f"activate_{t2.lower()}",   radius=40)
        cc[i,j] = np.corrcoef(x,y)[0,1]
    

In [ ]:
# look at correlation between Il33 and Activated Micro/Astro
x,y = compute_celltype_obs_correlation(adata_annot[adata_annot.obs.spatial_clust_annots=="CC"],  "Olig","Micro", f"activate_olig",f"activate_micro",   radius=30)
plt.figure(figsize=(5,5))
#plt.scatter(x,y,s=1)
plt.title(f"Olig -> Micro (R={np.corrcoef(x,y)[0,1]})")
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_olig_micro.pdf",bbox_inches='tight',dpi=300)

In [ ]:
# look at correlation between Il33 and Activated Micro/Astro
x,y = compute_celltype_obs_correlation(adata_annot[adata_annot.obs.spatial_clust_annots=="CC"],  "Olig","Astro", f"activate_olig",f"activate_astro",   radius=30)
plt.figure(figsize=(5,5))
#plt.scatter(x,y,s=1)
plt.title(f"Olig -> Astro (R={np.corrcoef(x,y)[0,1]})")
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()
#plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_olig_astro.pdf",bbox_inches='tight',dpi=300)

In [ ]:
adata_annot_yng = adata_annot[adata_annot.obs.age=='4wk']
adata_annot_old = adata_annot[adata_annot.obs.age=='90wk']


In [ ]:
# look at correlation between Il33 and Activated Micro/Astro
x,y = compute_celltype_obs_correlation(adata_annot_yng[adata_annot_yng.obs.spatial_clust_annots=="CC"],  "Olig","Micro", f"activate_olig",f"activate_micro",   radius=30)
plt.figure(figsize=(5,5))
#plt.scatter(x,y,s=1)
plt.title(f"Olig -> Micro (R={np.corrcoef(x,y)[0,1]})")
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_olig_micro_yng.pdf",bbox_inches='tight',dpi=300)

In [ ]:
# look at correlation between Il33 and Activated Micro/Astro
x,y = compute_celltype_obs_correlation(adata_annot_old[adata_annot_old.obs.spatial_clust_annots=="CC"],  "Olig","Micro", f"activate_olig",f"activate_micro",   radius=30)
plt.figure(figsize=(5,5))
#plt.scatter(x,y,s=1)
plt.title(f"Olig -> Micro (R={np.corrcoef(x,y)[0,1]})")
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_olig_micro_old.pdf",bbox_inches='tight',dpi=300)

In [ ]:
# look at correlation between Il33 and Activated Micro/Astro
x,y = compute_celltype_obs_correlation(adata_annot_old[adata_annot_old.obs.spatial_clust_annots=="CC"],  "Olig","Astro", f"activate_olig",f"activate_astro",   radius=30)
plt.figure(figsize=(5,5))
#plt.scatter(x,y,s=1)
plt.title(f"Olig -> Astro (R={np.corrcoef(x,y)[0,1]})")
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_olig_astro_old.pdf",bbox_inches='tight',dpi=300)

In [ ]:
# look at correlation between Il33 and Activated Micro/Astro
x,y = compute_celltype_obs_correlation(adata_annot_yng[adata_annot_yng.obs.spatial_clust_annots=="CC"],  "Olig","Astro", f"activate_olig",f"activate_astro",   radius=30)
plt.figure(figsize=(5,5))
#plt.scatter(x,y,s=1)
plt.title(f"Olig -> Astro (R={np.corrcoef(x,y)[0,1]})")
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()
plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_olig_astro_yng.pdf",bbox_inches='tight',dpi=300)

In [ ]:
x,y = compute_celltype_obs_correlation(adata_annot[adata_annot.obs.spatial_clust_annots=="CC"],  "Micro","Astro", f"activate_micro",f"activate_astro",   radius=30)
plt.figure(figsize=(5,5))
plt.title(f"Micro -> Astro (R={np.corrcoef(x,y)[0,1]})")
#plt.hist2d(x,y,cmap=plt.cm.viridis,bins=20,rasterized=True);
#plt.scatter(x,y,s=1)
sns.kdeplot(x=x,y=y,fill=True)
#plt.xlim([0,5])
#plt.axis('off')
sns.despine()

plt.savefig("/home/user/Dropbox/zhuang_lab/aging/aging_atlas_paper/figures_int/fig3_activation_corr_micro_astro.pdf",bbox_inches='tight',dpi=300)